In [1]:
import pandas as pd;
import json;
import math;

In [4]:
map_json = open('datamap.json');
nav = json.load(map_json)['raw'];

dataset = nav['files'];
pd.DataFrame(dataset).reindex(columns=['file','range'])

,file,range
0,SPX_WSJ.csv,"{'start': '2/1/1991', 'end': '3/11/2021'}"
1,SENSEX_WSJ.csv,"{'start': '1/1/1991', 'end': '3/11/2021'}"
2,N225_WSJ.csv,"{'start': '4/1/1991', 'end': '2/11/2021'}"
3,UKX_WSJ.csv,"{'start': '2/1/1991', 'end': '3/11/2021'}"
4,BTC-USD_Yahoo.csv,"{'start': '17/9/2014', 'end': '4/11/2021'}"


In [ ]:
stocks={}
names=['data_snp5', 'data_sens', 'data_n225', 'data_ftse', 'data_btco'];

for i in names:
    stocks[i] = pd.read_csv('clean/'+i+'.csv');
    stocks[i]